In [2]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import sparse
import random
from sklearn import tree
import math
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [29]:
#Fetching the mnist data
#Dividing it into test and train split
mnist = fetch_mldata('MNIST original')
X_train, X_test, y_train, y_test = train_test_split(mnist.data,mnist.target, test_size=0.15)

In [72]:
def getNewFeatures():
    features = {}
    data = mnist.data
    rectangles = getRectangles()
    for num in range(len(data)):
        if num%7000 == 0:
            print num
        da = np.reshape(data[num],(28,28))
        d = [[0 for i in range(28)]for j in range(28)]
        d[0][0] = 0
        for i in range(1,28):
            d[0][i] = d[0][i-1] + da[0][i]/128
        for i in range(1,28):
            d[i][0] = d[i-1][0] + da[i][0]/128

        for i in range(1,28):
            for j in range(1,28):
                d[i][j] = d[i][j-1] + d[i-1][j] - d[i-1][j-1] + da[i][j]/128
        
        feature=[]
        #Extracting left, right, upper and lower halves from the features
        for r in rectangles: 
            left_half = [[r[0][0],(r[0][1][0],(r[0][0][1]+r[0][1][1])/2)],[r[1][0],(r[1][1][0],(r[1][1][1]+r[1][0][1])/2)]]
            right_half = [[(r[0][0][0],(r[0][0][1]+r[0][1][1])/2),r[0][1]],[(r[1][0][0],(r[1][0][1]+r[1][1][1])/2),r[1][1]]]
            upper_half = getUpperhalf(r)
            lower_half = getLowerhalf(r)
            feature += [getBlackCount(left_half,d)-getBlackCount(right_half,d),getBlackCount(upper_half,d)-getBlackCount(lower_half,d)]
        features[num] = feature
    return features

In [73]:
def getRectangles():
    rectangles = []
    for i in range(9):
        for j in range(9):
            rectangles.append([[(i*2,j*2),(i*2,11+j*2)],[(11+i*2,j*2),(11+i*2,11+j*2)]])
    rect_19 = []
    
    for i in range(1000):
        rect_19.append([random.randint(0,27),random.randint(0,27),random.randint(0,27),random.randint(0,27)])
    final_rect =[]
    c =0
    for i in rect_19:
        if (math.sqrt(math.pow(i[0]-i[2],2) + math.pow(i[1]-i[3],2))) > 26.5:
            final_rect.append(i)
            c += 1
        if c == 19:
            break
    rect = []
    for r in final_rect:
        rect.append([(r[0],r[1]),(r[2],r[3]),(r[0],r[3]),(r[2],r[1])])
    def getKey(item):
        return item[0]
    for r in rect:
        r.sort(key=getKey)
    rectangles_temp = []
    for r in rect:
        rectangles_temp.append([r[:2],r[2:]])
    def getKey2(item):
        return item[1]
    for i in rectangles_temp:
        for j in range(2):
            i[j].sort(key=getKey2)
    print len(rectangles),len(rectangles_temp)
    return rectangles + rectangles_temp

In [74]:
def getBlackCount(r,d):
    return d[r[1][1][0]][r[1][1][1]] - d[r[0][1][0]][r[0][1][1]] - d[r[1][0][0]][r[1][0][1]] + d[r[0][0][0]][r[0][0][1]]

In [75]:
def getUpperhalf(rect_current):
    t = rect_current
    new_height = (t[0][0][0]+t[1][0][0])/2
    return [[t[0][0],t[0][1]],[(new_height,t[1][0][1]),(new_height,t[1][1][1])]]

def getLowerhalf(t):
    new_height = (t[0][0][0]+t[1][0][0])/2
    return [[(new_height,t[0][0][1]),(new_height,t[0][1][1])],[t[1][0],t[1][1]]]

In [76]:
features = getNewFeatures()

81 19
0
7000
14000
21000
28000
35000
42000
49000
56000
63000


In [59]:
print features[10], features[10000]

[-6, -6, -17, -17, -29, -25, -26, -30, -16, -38, -1, -41, 8, -32, 20, -22, 23, -15, -10, -10, -24, -18, -36, -18, -25, -15, -13, -19, -2, -18, 3, -7, 22, -4, 29, -5, -17, -13, -31, -13, -37, -5, -18, 4, -11, 1, -6, 8, -3, 13, 24, 8, 35, -1, -24, -12, -33, -5, -27, 3, -3, 7, -2, 8, -6, 16, -9, 13, 21, 5, 32, -2, -29, -11, -30, -2, -12, 2, 11, -1, 0, 6, -12, 14, -14, 6, 20, 0, 30, -2, -34, -8, -29, -5, -3, -9, 15, -17, -1, -3, -13, 3, -7, -3, 22, -2, 27, 3, -39, -7, -32, -12, -2, -20, 15, -27, 0, -14, -4, -10, 4, -8, 23, -1, 21, 7, -35, -1, -29, -9, -2, -18, 9, -19, 0, -12, 3, -11, 12, -4, 21, 5, 15, 11, -27, 7, -23, 3, -4, 2, 1, 9, -1, 7, 7, 5, 17, 9, 18, 12, 10, 10, 31, -15, -24, -2, 17, -7, 15, -7, -22, -16, 53, -19, 0, 12, 71, 29, 23, -7, 23, -21, 40, 16, -40, -14, -70, -18, -8, 2, 25, 11, 15, 3, 12, -10, -13, -7, -26, -24] [0, 0, 0, 0, -2, -2, -9, -9, -19, -15, -18, -14, -4, -14, 16, -14, 17, -9, 0, 0, -1, -1, -7, -7, -15, -15, -23, -13, -14, -10, 2, -10, 19, -7, 17, 1, 0, 0, -4, -4

In [77]:
new_data = []
for num in range(len(features)):
    new_data.append(features[num])
clf = LogisticRegression()
clf.fit(new_data[:62000],mnist.target[:62000])
score = clf.score(new_data[62000:],mnist.target[62000:])

In [78]:
print "The score of Linear regression with HAAR features of 200 is : ",score

The score of Linear regression with HAAR features of 200 is :  0.897


In [79]:
clf_tree = tree.DecisionTreeClassifier()
clf_tree.fit(new_data[:62000],mnist.target[:62000])
score = clf_tree.score(new_data[62000:],mnist.target[62000:])
print score

0.858375


In [67]:
score

0.81379999999999997